In [ ]:
import math
import torch
import torch.nn as nn
import pandas as pd

In [ ]:
path = 'C:/Users/jeanb/Documents/Python Scripts/ML-intro-main/datasets/'
data = pd.read_csv(path + 'iris.csv')
data.shape # 150, 5
data.keys() # 'sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species'
features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
label = 'species'
train_idx = list(range(0, 40)) + list(range(50, 90)) + list(range(100, 140))
val_idx = list(range(40, 50)) + list(range(90, 100)) + list(range(140, 150))
type(data['sepal_length'][val_idx])

# from text labels to id labels
label_idx = {}
idx = 0
for i in range(data.shape[0]):
    label_name = data[label][i]
    if label_name not in label_idx:
        label_idx[label_name] = idx
        idx += 1
print(label_idx)

{'setosa': 0, 'versicolor': 1, 'virginica': 2}


In [35]:
X_train = torch.zeros((len(train_idx), len(features)))
y_train = torch.zeros(len(train_idx))

X_val = torch.zeros((len(val_idx), len(features)))
y_val = torch.zeros(len(val_idx))

for i, k in enumerate(features):
    X_train[:, i] = torch.tensor(data[k][train_idx].values, dtype=torch.float32)
    X_val[:, i] = torch.tensor(data[k][val_idx].values, dtype=torch.float32)

for i, j in enumerate(train_idx):
    idx = label_idx[ data[label][j] ]
    y_train[i] = torch.tensor(idx)

for i, j in enumerate(val_idx):
    idx = label_idx[ data[label][j] ]
    y_val[i] = torch.tensor(idx)

# Normalization
X_mean = X_train.mean(dim=0)
X_train /= X_mean
X_val /= X_mean

print("Training:", X_train.shape, y_train.shape)
print("Evaluation:", X_val.shape, y_val.shape)

Training: torch.Size([120, 4]) torch.Size([120])
Evaluation: torch.Size([30, 4]) torch.Size([30])


In [45]:
# Synthetic dataset

N = 200
centers = torch.Tensor([[-1., -1.], [1., 1.]])
X_train = torch.randn((N, 2))*0.2
X_train[:N//2] += centers[0]
X_train[N//2:] += centers[1]
# shuffling
X_train = X_train[torch.randperm(N)]


In [46]:
class GaussianMixture():
    """
    GMM Gaussian Mixture Model for clustering
    """
    def __init__(self, K:int, X_train):
        assert type(K) == int and K > 0
        assert len(X_train.shape) == 2
        self.K = K # number of clusters
        self.X_train = X_train
        self.N, self.x_dim = X_train.shape
        self._init()

    def _init(self):
        """Initialize the mixing coefficients, mean and variances"""
        self.mixing = torch.ones(self.K)/self.K
        # initialize means by taking mean over subsets of the dataset
        self.mean = torch.zeros((self.K, self.x_dim))
        self.covariance = torch.zeros((self.K, self.x_dim, self.x_dim))
        self.det_cov = torch.zeros(self.K) # determinant of covariance matrices
        self.inv_cov = torch.zeros((self.K, self.x_dim, self.x_dim)) # inverse of covariance matrices
        for k in range(self.K):
            i = k*self.N//self.K
            j = (k+1)*self.N//self.K
            self.mean[k] = self.X_train[i:j].mean(dim=0)
            self.covariance[k] = torch.cov(self.X_train[i:j].T)
            self.det_cov[k] = torch.linalg.det(self.covariance[k])
            self.inv_cov[k] = torch.linalg.inv(self.covariance[k])


    def _responsabilities(self):
        """Calculate the responsibilities for each datapoint based on the kernels"""
        self.pdf = torch.zeros((self.K, self.N)) # N(x_i, mu_k, sigma_k)
        for k in range(self.K):
            self.pdf[k] = self._Gaussian(k)
        
        # one for each datapoint for each cluster
        self.responsibility = (self.mixing.T * self.pdf.T).T # K, N
        self.responsibility /= self.responsibility.sum(dim=0) # sum over K
        self.sum_responsability = torch.sum(self.responsibility, dim=1) # sum over N

    def _Gaussian(self, k):
        """Returns the Gaussian probability  P( X_train | mu_k sigma_k )
        for cluster k """
        mu = self.mean[k]
        p = (self.X_train - mu) @ self.inv_cov[k] @ (self.X_train - mu).T
        d = torch.sqrt( (2*math.pi)**self.x_dim * self.det_cov[k])
        return torch.exp(-0.5*torch.diag(p)) / d
    
    def _update(self):
        """Update the mean, covariance and mixing coefficients"""
        self.mean = self.responsibility @ self.X_train # K, N @ N, x = K, x
        self.mean = (self.mean.T / self.sum_responsability).T
        self.mixing = self.sum_responsability/self.N

        for k in range(self.K):
            self.covariance[k] = torch.sum(self.responsibility[k] @ (self.X_train - self.mean[k]) @ (self.X_train - self.mean[k]).T)
            self.covariance[k] /= self.sum_responsability[k]
            self.det_cov[k] = torch.linalg.det(self.covariance[k])
            self.inv_cov[k] = torch.linalg.inv(self.covariance[k])

    def train(self, epochs:int):
        for e in range(epochs):
            self._responsabilities()
            self._update()


    def predict(self, X:torch.Tensor):
        """X: dim (batch, number_of_features)"""
        assert len(X.shape) == 2 
        assert X.shape[1] == self.nb_features

        proba = []
        classes = torch.zeros(X.shape[0])
        for sample_id in range(X.shape[0]):
            proba.append( [] )
            for c in self.class_probability.keys():
                proba[sample_id].append(self._Gaussian_product(X[sample_id], c))
            proba[sample_id] = torch.hstack(proba[sample_id])
            classes[sample_id] = torch.argmax(proba[sample_id]) # Maximum A Posteriori
        print(proba)
        return classes

# Shuffle dataset
X_train = X_train[torch.randperm(X_train.shape[0])]
K = 5
model = GaussianMixture(K=K, X_train=X_train)
model.train(epochs=1)

_LinAlgError: linalg.inv: The diagonal element 2 is zero, the inversion could not be completed because the input matrix is singular.

In [ ]:
# model.X_train.shape, model.inv_cov[0].shape
# model.mixing.shape, model.pdf.shape
# (model.mixing.T * model.pdf.T).T.shape
# model.mean.shape, model.sum_responsability.T.shape
# (model.mean.T / model.sum_responsability).shape
# model.covariance
# model.mean
# model.det_cov
# torch.cov(X_train[0:40].T)
# model.responsibility


tensor([6, 7, 8, 4, 5, 0, 2, 9, 3, 1])

In [44]:
## TODO
# # Make predictions on the training set
# predictions = model.predict(X_train)

# # Calculate accuracy
# accuracy = (predictions == y_train).float().mean()
# print(f"Training Accuracy: {accuracy.item() * 100:.2f}%")